In [1]:
import tensorflow as tf
import keras_cv
import tensorflow_datasets as tfds
import keras

In [2]:
BATCH_SIZE = 16
NUM_CLASSES = 3
NUM_CLASSES_WASTE = 6
IMAGE_SIZE = (180, 180)

In [3]:
# Create a preprocessing pipeline with augmentations

augmenter = keras_cv.layers.Augmenter(
    [
        keras_cv.layers.RandomFlip(),
        keras_cv.layers.RandAugment(value_range=(0, 255)),
        keras_cv.layers.CutMix(),
    ],
)

def preprocess_data(images, labels, augment=False):
    labels = tf.one_hot(labels, NUM_CLASSES)
    inputs = {"images": images, "labels": labels}
    outputs = inputs
    if augment:
        outputs = augmenter(outputs)
    return outputs['images'], outputs['labels']



In [4]:
rps_train_dataset, rps_test_dataset = tfds.load(
    'rock_paper_scissors',
    as_supervised=True,
    split=['train', 'test'],
)

rps_train_dataset = rps_train_dataset.batch(BATCH_SIZE).map(
    lambda x, y: preprocess_data(x, y, augment=True),
        num_parallel_calls=tf.data.AUTOTUNE).prefetch(
            tf.data.AUTOTUNE)
rps_test_dataset = rps_test_dataset.batch(BATCH_SIZE).map(
    preprocess_data, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
        tf.data.AUTOTUNE)


In [6]:
# Create a model using a pretrained backbone
rps_backbone = keras_cv.models.EfficientNetV2Backbone.from_preset(
    "efficientnetv2_b0_imagenet"
)
rps_model = keras_cv.models.ImageClassifier(
    backbone=rps_backbone,
    num_classes=NUM_CLASSES,
    activation="softmax",
)
rps_model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

In [7]:
# Train your model
rps_model.fit(
    rps_train_dataset,
    validation_data=rps_test_dataset,
    epochs=1,
)